In [5]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account

In [6]:
# Nodes runing with POW
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))

# Loading EV
load_dotenv

# Loading Mnemonic EV and (set this mnemonic as an environment variable, and include the one you generated as a fallback using)
mnemonic = os.getenv('MNEMONIC', "mango truly useless stand leopard bracket decade deputy message test kit used")
print(mnemonic)

mango truly useless stand leopard bracket decade deputy message test kit used


## Creating functions to transact

In [7]:
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys

In [8]:

#Setting dictionary of coins to be used in the wallet

coins = {"eth", "btc-test", "btc"}
numderive = 3

In [9]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [10]:
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc"
"cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt"


In [11]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "1Hk26uv3aSo2WmKNPbeUGjNWPFDBLU2waW",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KzQ3o74J1R3MFuzvRNQWjckRz3AYZHgggJRbs3P7TH1ZoXJqKety",
            "pubkey": "0399c9a8f6933934bfe811fc524c0e8684f971819398a0581f20ddf6371db0bb18",
            "pubkeyhash": "b7a4d19972d97725ebe1d980ac875f4b41322ec5",
            "xprv": "xprvA4EtnVRNTKMCodewrh6ipRZg96w7tyJMntxXmtPitFkMn7Dwe6BRfeC2bnL8DRs96stL1Kn752eZJqfFX1Fs25uaRxnUSSShU91qzeg5gWK",
            "xpub": "xpub6HEFBzxGHguW27jQxidjBZWQh8mcJS2DA7t8aGoLSbHLeuZ6BdVgDSWWT5kbpKZxGjjA8qDfgYCyKgBeau5Qp48c5KC9dTaYjWd3jLfF8MG"
        },
        {
            "address": "16LsmsW5nshaMWz4XXE9qZjE1HHEEF61Ma",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L3PRGxRUzyYGDEnWsdtyunpa5DCfYcgKfBSCCXGEHofUyxYj7zc6",
            "pubkey": "03412c487c7a81706519d3236215f717b9134a39e0447e1a70de08fd65d83ab7e1",
            "pubkeyhash":

In [12]:
def priv_key_to_account(coin, priv_key):
    """Convert the privkey string in a child key to an account object that bit or web3.py can use to transact"""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

In [13]:
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD>


In [14]:
def create_trx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global trx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [15]:
def send_trx(coin, account, recipient, amount):
    """call create_trx, sign the transaction, then send it to the designated network"""
    if coin == "eth": 
        trx_eth = create_trx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_trx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

## Calling the functions to create, sign and execute transactions (BTCTest and ETH)

### BTC test transactions

In [17]:
# create BTC transaction
create_trx(BTCTEST,btc_acc,"miZgMxdGzSxCTpWazfD2KqhewoUvcQ6CC1", 0.0001)

'{"unspents":[{"amount":407615,"confirmations":189,"script":"76a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac","txid":"acafe34cff3ec45e1201e6d477a6a4170dbb66074ac39838be4e3da16eb6a8d5","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["miZgMxdGzSxCTpWazfD2KqhewoUvcQ6CC1",10000],["mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD",374563]]}'

In [18]:
#send BTC transaction
send_trx(BTCTEST,btc_acc,'miZgMxdGzSxCTpWazfD2KqhewoUvcQ6CC1',0.0001)

'0100000001d5a8b66ea13d4ebe3898c34a0766bb0d17a4a677d4e601125ec43eff4ce3afac010000006b483045022100c592093053cc6f218f45fb2e8a775d698e1482c7c1ff7abc6069f9beefa297f80220779db33cdfcb8466baee9709b11926832a2e00f44d477e9cf4cd20759e8ab7a8012102dcb7a11c7172966e58679c0cb207bfb95d119cc55f9fb318d0a522aaff5b9577ffffffff0210270000000000001976a914216c53486c93fa265d93a5ea85b6d3499b136d9188ac23b70500000000001976a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac00000000'

### ETH transactions

In [19]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [20]:
from web3 import Web3, HTTPProvider

In [21]:
#connecting to HTTP with address pk
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x5619d0cce54919511eb9f3b678de07bd346044acee0bdd6cdf4497ba45e03f81"))

In [22]:
#checking the Block Number
w3.eth.blockNumber

31

In [23]:
# double check if connected to blockchain. 
w3.isConnected()

True

In [43]:
w3.eth.getBalance("0xdeA9cE1d814f6dAaEe900D864B7f1974Ab2f0A44")

0

In [47]:
create_trx(ETH,eth_acc,"0x4BDF6bE01daedb03c89E4c9CaAB4878E989Ade2F", 0)

{'to': '0x4BDF6bE01daedb03c89E4c9CaAB4878E989Ade2F',
 'from': '0x02194a55DDA1B029F70220D53f099F38c629ab36',
 'value': 0,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}